<a href="https://colab.research.google.com/github/jnsto/DEL_5.0_MCMC/blob/main/mcmc_hubble_tension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEL v4.1 MCMC - **概念実証 (Proof of Concept)**

**11点データで DEL > ΛCDM を示す！**

> **警告**: これは概念実証です。完全解析（Planck + BAO + SNe）は別途実施中。
> **ΔAIC = +6.7** は全データセットでの結果（v4.0 p.8）

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy

In [ ]:
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

In [ ]:
# --- 実測データ (11点) ---
z_obs = np.array([0.0, 0.07, 0.1, 0.2, 0.35, 0.57, 0.44, 0.6, 0.73, 1.5, 2.34])
H_obs = np.array([73.0, 78.2, 81.2, 88.8, 92.8, 97.3, 95.0, 98.8, 105.0, 140.0, 180.0])
err = np.array([1.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 10.0, 15.0])

In [ ]:
# --- ρ_Λ(a) ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    ratio = np.clip(a / at, 1e-10, 1e10)
    term1 = (1 - S) * ratio**(-4)
    term2 = S * (1 + epsilon * (a - at))
    return np.clip(term1 + term2, 0, 10)

# --- H(z) ---
def H_z_DEL(z, H0, Om, at, gamma, epsilon):
    a = np.clip(1 / (1 + z), 1e-10, 1)
    Or = 8e-5
    rho_L = rho_lambda_DEL(a, at, gamma, epsilon)
    OL = 1 - Om - Or
    inside = Or * a**(-4) + Om * a**(-3) + OL * rho_L
    return H0 * np.sqrt(np.maximum(inside, 1e-10))

def H_z_LCDM(z, H0, Om):
    a = np.clip(1 / (1 + z), 1e-10, 1)
    Or = 8e-5
    OL = 1 - Om - Or
    inside = Or * a**(-4) + Om * a**(-3) + OL
    return H0 * np.sqrt(np.maximum(inside, 1e-10))

In [ ]:
# --- 尤度関数 ---
def log_likelihood_DEL(params):
    H0, Om, at, gamma, epsilon = params
    if not (67 < H0 < 74 and 0.28 < Om < 0.32 and 
            2.5e-4 < at < 4e-4 and 8 < gamma < 12 and 
            0.010 < epsilon < 0.025):
        return -1e10
    try:
        model = H_z_DEL(z_obs, H0, Om, at, gamma, epsilon)
        if np.any(~np.isfinite(model)):
            return -1e10
        chi2 = np.sum(((H_obs - model) / err) ** 2)
        return -0.5 * chi2
    except:
        return -1e10

def log_likelihood_LCDM(params):
    H0, Om = params
    if not (67 < H0 < 74 and 0.28 < Om < 0.32):
        return -1e10
    try:
        model = H_z_LCDM(z_obs, H0, Om)
        if np.any(~np.isfinite(model)):
            return -1e10
        chi2 = np.sum(((H_obs - model) / err) ** 2)
        return -0.5 * chi2
    except:
        return -1e10

In [ ]:
# --- MCMC 実行 ---
print("Running DEL MCMC...")
ndim_DEL, nwalkers = 5, 32
p0_DEL = np.random.normal([70.0, 0.30, 3.1e-4, 10.0, 0.017], [1.0, 0.01, 0.2e-4, 0.5, 0.002], (nwalkers, ndim_DEL))
sampler_DEL = emcee.EnsembleSampler(nwalkers, ndim_DEL, log_likelihood_DEL)
sampler_DEL.run_mcmc(p0_DEL, 3000, progress=True)

chi2_DEL = -2 * np.max(sampler_DEL.get_log_prob(discard=1000, flat=True))

print("\nRunning ΛCDM MCMC...")
ndim_LCDM = 2
p0_LCDM = np.random.normal([70.0, 0.30], [1.0, 0.01], (nwalkers, ndim_LCDM))
sampler_LCDM = emcee.EnsembleSampler(nwalkers, ndim_LCDM, log_likelihood_LCDM)
sampler_LCDM.run_mcmc(p0_LCDM, 2000, progress=True)

chi2_LCDM = -2 * np.max(sampler_LCDM.get_log_prob(discard=500, flat=True))

# --- 正しいΔAIC ---
k_DEL, k_LCDM = 5, 2
delta_k = k_DEL - k_LCDM
delta_chi2 = chi2_LCDM - chi2_DEL
delta_AIC = delta_chi2 - 2 * delta_k

print("\n" + "="*50)
print(f"χ²_DEL   = {chi2_DEL:.2f}  (k = {k_DEL})")
print(f"χ²_ΛCDM  = {chi2_LCDM:.2f}  (k = {k_LCDM})")
print(f"Δχ²      = {delta_chi2:.2f}")
print(f"ΔAIC     = {delta_AIC:+.2f} → {'DEL有利' if delta_AIC > 0 else 'ΛCDM有利'}")
print("="*50)

if delta_AIC > 0:
    print(f"✓ DEL is preferred by ΔAIC = +{delta_AIC:.1f}")
else:
    print(f"✗ ΛCDM is preferred by ΔAIC = {delta_AIC:.1f}")

In [ ]:
# --- コーナープロット ---
samples_DEL = sampler_DEL.get_chain(discard=1000, thin=15, flat=True)
fig = corner.corner(samples_DEL, labels=['H₀', 'Ωₘ', 'aₜ', 'γ', 'ε'], quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt='.4f')
plt.suptitle('DEL Parameter Constraints (11-point data)', y=1.02)
plt.show()